## On-policy prediction with approximation

### Value-function Approximation （价值函数近似法）


- update notation $s \mapsto u$，其中 $s$ 是更新后的状态，$u$ 是 $s$ 的估计价值函数所要达到的目标:

    - **MC**: $S_t \mapsto G_t$
    - **TD(0)**: $S_t \mapsto R_{t+1} + \gamma \hat{v}(S_{t+1}, \mathbf{w}_t)$
    - **n-step TD**: $S_t \mapsto G_{t:t+n}$
    - **DP**: $s \mapsto \mathbb{E}\left[R_{t+1} + \gamma \hat{v}(S_{t+1}, \mathbf{w}_t) \middle| S_t = s\right]$
    

- Function Approximation 函数近似法使用 $s \mapsto g$ 近似 $u$ 。


- 表格式每次更新当前状态，其他状态不更新；近似法会对当前状态和其他部分或全部状态同时更新。

### The Prediction Objective($\overline{VE}$) 预测目标


- 近似法的更新会同时影响多个状态，所以需要确定状态的重要程度。定义状态的一种分布：$\mu(s) \ge 0, \sum_s \mu(s) = 1$，分布表示状态对损失（error）的重视程度。（通常使用时间步数的分数表示）如果是 on-policy 训练则称这种分布为 on-policy 分布。


- 损失函数定义为估计价值函数与真实价值函数差的平方的数学期望（均方误差）：


$$\overline{VE}(\mathbf{w}) \dot= \sum_{s\in \mathcal{S}} \mu(s) \left[v_{\pi}(s) - \hat{v}(s, \mathbf{w})\right]^2$$


- 片段式任务的 on-policy 分布：


$$\eta(s) = h(s) + \sum_\bar{s}\eta(\bar{s})\sum_a \pi(a|\bar{s})p(s|\bar{s},a), \forall{s} \in \mathcal{S}$$

其中： $h(s)$ 是 $s$ 为片段的初始状态的概率，$\eta(s)$ 表示到达 $s$ 的时间步数。

可知 on-policy 分布:


$$\mu(s)=\frac{\eta(s)}{\sum_{s'}\eta(s')}, \forall{s} \in \mathcal{S}$$

### Stochastic-gradient and Semi-gradient Methods 随机梯度和半梯度法


- 随机梯度下降 （SGD）方法是通过在每个示例之后对权重向量进行少量调整，以最大程度地减少误差


$$\mathbf{w}_{t+1} \dot= \mathbf{w}_t - \frac{1}{2} \alpha \nabla \left[v_{\pi}(S_t) - \hat{v}(S_t,\mathbf{w}_t)\right]^2 = \mathbf{w}_t + \alpha \left[v_{\pi}(S_t) - \hat{v}(S_t, \mathbf{w}_t)\right] \nabla \hat{v}(S_t, \mathbf{w}_t)$$


其中, $\alpha$ 是一个正数，导数向量 $\nabla f(\mathbf{w})$ 称为 $f$ 关于 $\mathbf{w}$ 的梯度，是误差下降最快的方向。


- 确定监督学习的目标(随机梯度)


因为 $v_\pi(S_t)$  不知道,此时假定 $U_t \in \mathbb{R}$ 是 $v_\pi(S_t)$ 的无偏估计,即 $\mathbb{E}[U_t|S_t=s] = v_\pi(S_t)$ ,算法仍然可以收敛.定义:


$$\mathbf{w}_{t+1} \dot= \mathbf{w}_t + \alpha \left[U_t - \hat{v}(S_t, \mathbf{w}_t)\right] \nabla \hat{v}(S_t, \mathbf{w}_t)$$


以 MC 算法为例,可以定义 $U_t \dot= G_t$,算法伪码:


![gradient_MC](../images/gradient_MC.png)


- 确定监督学习的目标(半梯度)


如果 $U_t$ 是有偏估计, 因为估计值与 $\mathbf{w}_t$ 有关,因此算法只会在部分梯度中有效,所以称为**半梯度法**.


以 TD(0)算法为例, 可以定义 $U_t \dot=  R_{t+1} + \gamma \hat{v}(S_{t+1}, \mathbf{w})$,算法伪码:


![semi_gradient_TD0](../images/semi_gradient_TD0.png)

### Linear Methods


- 价值函数的线性近似定义


$$\hat{v}(s, \mathbf{w}) \dot= \mathbf{w}^\top \mathbf{x}(s) \dot= \sum^{d}_{i=1}w_i x_i(s)$$


其中 $\mathbf{x}(s)$ 是状态 $s$ 的特征矢量, $x_i: \mathcal{S}\to \mathbb{R}$.


- 线性 SGD


线性近似函数关于 $\mathbf{w}$ 的梯度: $\nabla \hat{v}(s, \mathbf{w}) = \mathbf{x}(s)$. 因此梯度更新形式:


$$\mathbf{w}_{t+1} \dot= \mathbf{w}_t + \alpha \left[U_t - \hat{v}(S_t, \mathbf{w}_t)\right] \mathbf{x}(S_t)$$


随机梯度下降可以保证收敛到最优,但是半梯度法需要特定公式,以 TD(0) 算法为例:


$$\mathbf{w}_{t+1} \dot= \mathbf{w}_t + \alpha \left(R_{t+1} + \gamma \mathbf{w}_t^\top \mathbf{x}_{t+1} - \mathbf{w}_t^\top \mathbf{x}_t \right) \mathbf{x}_t = \mathbf{w}_t + \alpha \left(R_{t+1} \mathbf{x}_t - \mathbf{x}_t(\mathbf{x}_t - \gamma \mathbf{x}_{t+1})^\top \mathbf{w}_t \right)$$


其中定义 $\mathbf{x}_t = \mathbf{x}(S_t)$. 算法期望的下一个权重可以写成:


$$\mathbb{E}[\mathbf{w}_{t+1}|\mathbf{w}_t] = \mathbf{w}_t + \alpha (\mathbf{b} - \mathbf{A} \mathbf{w}_t)$$


算法如果收敛,假设此时权重是 $\mathbf{w}_{TD}$,那么:


$$\mathbf{b} - \mathbf{A}  \mathbf{w}_{TD} = 0 \Rightarrow \mathbf{b} = \mathbf{A} \mathbf{w}_{TD} \Rightarrow \mathbf{w}_{TD} \dot= \mathbf{A}^{-1} \mathbf{b}$$


上述等式结果称为 **TD固定点**,  TD(0)算法收敛于此.